# Assignment 01a - Data Visualization


## Goals
You can load and visualize measurements from the dataset:
  * you can visualize the coordinate frames of the vehicle mounted sensors (3D axes of camera, LiDAR, radar)
  * **MP only**: you can visualize LiDAR point clouds in the camera image
  * **MP only**: you can visualize radar measurements in the camera image
  * you can visualize the vehicle pose over time in a world-static frame (3D axes)

In addition, you will be able to work with
  * annotated 3D object labels (such as projecting them into the camera image)
  * ground plane models

We provide you with a dataset recorded from a vehicle with different sensors (camera, LiDAR, radar, pose (odometry)) which provide measurements over time. Each sensor lives in a coordinate frame which defines its rotation and translation with respect to another frame, e.g., how are camera and LiDAR positioned relative to each other.

Please make use of the classes for dataset access you got familiar with in the practica, namely `Dataset`, `Sequence` and `Measurements`, and the functions you implemented in Practicum 1, such as `project_points(camera_projection_matrix, points)`.
There is also a recap in this notebook. 
We can import it within this notebook using `from ipynb.fs.defs.practicum1 import project_points`.
If there are technical issues (e.g. with `ipynb` import, you can also copy over code from your practica, but please mark it as such.

## Input
Custom sequence from dataset with defined `start_index` and `end_index`

## Output
Plots and visualizations within this notebook.
No additional files.

## Grading
You will get credits for the correct visualizations and answers.
Most answers will be autograded, so please stick to the instructions.

In [ ]:
# CELL REF: C1.0a
# some magic to ease implementing within jupyter notebooks
from IPython import get_ipython

ipython = get_ipython()
if ipython:
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

The next cell loads the variable `COURSE_TYPE` that you configured in `fa_00_overview.ipynb`. This notebook assignment contains parts that are MP and IV-specific. Using this variable, certain parts of this notebook are automatically skipped.

In [ ]:
# CELL REF: C1.0b
from ipynb.fs.defs.fa_00_overview import COURSE_TYPE, RUN_ALL
print('Course type:', COURSE_TYPE)
print('Run both MP and IV implementations:', RUN_ALL)

In [ ]:
# CELL REF: C1.0c
assert COURSE_TYPE == 'IV' or COURSE_TYPE == 'MP'

# Recap on the classes `Dataset`, `Sequence`, `Measurements`
You will work with data from the View-of-Delft dataset. This dataset was recently recorded in Delft from a moving vehicle equipped with various sensors. It was collected by researchers from the Intelligent Vehicles group at TU Delft. You will work with a subset of the dataset: A short sequence and only the stereo camera, LiDAR, and radar sensors.

You have already worked with the sequence loader in practicum 1, but still we will recap some of the concepts. 

* The `Dataset` class encapsulates all measurements of all sensors over time for the View-of-Delft dataset
* The `Sequence` is a subset of the dataset encapsulating all measurements of all sensors over time for a subset of time (defined by `start_index` and `end_index`).
* The `Measurements` class encapsulates the measurements of all sensors for a single point in time.

See the code below for the an exemplaric use and the available getters of the `Measurements` class.

In [ ]:
# CELL REF: C1.0d
from common.sequence_loader import Dataset

dataset = Dataset()  # the view-of-delft dataset
sequence = dataset.get_custom_sequence(1430, 1431)  # a subset of two timesteps of the dataset
measurements = next(iter(sequence))  # first measurements object within the sequence

# show getters of dataset
sorted([method for method in dir(measurements) if method.startswith("get_")])

* The `get_T_*` methods return homogeneous transforms between sensor frames according to the conventions recapitulated further below.
* The `get_*_points` methods return points in 4xN homogenenous coordinates
* The `get_*_image` methods return the camera images represented as NumPy arrays
* The `get_camera_projection_matrix` methods returns the 3x4 projection matrix for the left camera
* The `get_right_camera_projection_matrix` methods returns the 3x4 projection matrix for the right camera
* The `get_disparity` method returns the disparity map (based on stereo images)
* The `get_ground_plane` method returns the ground plane in the LiDAR frame and will be covered later in this notebook.

Feel free to explore the data obtained by the other getters.

## Recap: Extrinsic calibration

The camera, LiDAR and radar data for a particular `Measurement` are recorded simultaneously.
Still, we cannot plot the the camera, LiDAR and radar points in the same plot in a reasonable manner, because the sensors, and thereby the frame of reference for the point clouds, are at different locations.
Thus, we need to take the measurement's frame of reference into account.

The sensors are rigidly attached to the vehicle and the relative poses between the sensors do not change over time.
The static transformations between the camera, LiDAR and radar were obtained using an extrinsic calibration procedure.
Note that the transformations are given as 4x4 matrices for homogenous coordinates and points are given in `[x, y, z, 1]` homogeneous coordinates.

We will be working with transformations between different sensors and transformations between different points in time.
It is easy to lose track of which data is in what frame, so we start with establishing some conventions.


## Recap: Transform naming conventions
`pc_frameA`: point cloud (`pc`) given in frame `frameA` with shape (4, N) (homogeneous)

`pc_frameB`: point cloud given in frame `frameB` with shape (4, N) (homogeneous)

Convention: `T_frameB_frameA` with shape (4, 4) transforms `pc_frameA` into `frameB`: <br/>
`pc_frameB = T_frameB_frameA.dot(pc_frameA)`

You could also say, `frameA` gets cancelled out by `T_frameB_frameA`. This convention reads well if you apply multiple transformations:<br/>
`T_frameC_frameA = T_frameC_frameB.dot(T_frameB_frameA)`

Taking the inverse of the transformation matrix allows you to reverse the transformation:<br/>
`T_frameB_frameA = np.linalg.inv(T_frameA_frameB)`

See the code below for some hands-on transform juggling between the LiDAR and the radar frames:

In [ ]:
# CELL REF: C1.0e
import numpy as np

T_cam_lidar = measurements.get_T_camera_lidar()  # homogeneous transform
T_cam_radar = measurements.get_T_camera_radar()
T_lidar_cam = np.linalg.inv(T_cam_lidar)
T_lidar_radar = np.dot(T_lidar_cam, T_cam_radar)
T_radar_cam = np.linalg.inv(T_cam_radar)
T_radar_lidar = np.dot(T_radar_cam, T_cam_lidar)
T_radar_lidar  # homogeneneous transform

## Recap on homogeneous transformation matrices
See the code below for a recap on the content of a homogeneous transformation matrix and which computation of rotation and translation it combines within one operation.

In [ ]:
# CELL REF: C1.0f
# rotation matrix is top left 3x3 sub-matrix of homogeneous matrix
R_cam_lidar = T_cam_lidar[0:3, 0:3]
# translation vector is rightmost column of homogeneous matrix
# it reflects which point within the cam frame denotes the origin of the LiDAR frame.
# this is also what you get, when you do T_cam_lidar.dot([0,0,0,1]) 
t_cam_lidar = T_cam_lidar[0:3, 3]

# get random point in LiDAR frame (in homogeneous coordinates)
p_lidar = np.random.random(4)
p_lidar[3] = 1.0  # make homogeneous

# transform LiDAR point to camera frame via homogeneous matrix
p_cam1 = T_cam_lidar.dot(p_lidar)

# transforma LiDAR point to camera frame 'manually' via rotation matrix and translation vector
p_cam2 = R_cam_lidar.dot(p_lidar[:3]) + t_cam_lidar

# both computation ways yield the same point in camera frame
assert np.allclose(p_cam1[:3], p_cam2)

## 1.1 Dataset size
Please work with the custom `Sequence` of `Dataset` with `start_index=1430` and `end_index=1545`. How many measurement timesteps does it consist of? Please set the variable `n_timesteps` to the corresponding size based on the `Sequence` object you instantiate.

In [ ]:
# CELL REF: C1.1a
from common.sequence_loader import Dataset

start_index = 1430
end_index = 1545

n_timesteps = 0  # fill me with the right value
sequence = None  # instantiate me from the Dataset

# YOUR CODE HERE

n_timesteps = end_index - start_index + 1
sequence = dataset.get_custom_sequence(start_index, end_index)

# raise NotImplementedError()

In [ ]:
# CELL REF: C1.1b
# DO NOT DELETE THIS CELL!

## 1.2 Coordinate frames
Please use `k3d.plot` to create a 3D visualization with the coordinate axes of `camera`, `lidar` and `radar`.
Goal is to see how the sensors are located relative to each other.
Plot the three axes in the camera frame.

Use `plot_axes` from `common.k3d_helpers` to visualize the coordinate axes.
Use `k3d.text` to give the names `radar`, `camera` and `lidar` to the axes.
For simplicity, we take the first measurements object of the sequence.

In [ ]:
# CELL REF: C1.2a
from common.k3d_helpers import plot_axes

# you can show the doc of the plot_axes function
# via the builtin help() function
#
# or just put your cursor into the brackets of plot_axes()
# and press shift-tab (multiple times)
help(plot_axes)

In [ ]:
# CELL REF: C1.2b
import k3d
import numpy as np
from common.k3d_helpers import plot_axes

# use first measurement
measurements = next(iter(sequence))

# get transforms
T_camera_lidar = measurements.get_T_camera_lidar()
T_camera_radar = measurements.get_T_camera_radar()

# create plot
plot = k3d.plot(camera_auto_fit=False)

# YOUR CODE HERE

# For the plot of the camera coordinate axes I feed an identity matrix to the algorithm
plot += plot_axes(T_plotorigin_target=np.eye(4).astype(np.float32), length=1.0)
# For the plot of the lidar and radar coordinate axes I use the two transformation matrices already given
plot += plot_axes(T_plotorigin_target=T_camera_lidar.astype(np.float32), length=1.0)
plot += plot_axes(T_plotorigin_target=T_camera_radar.astype(np.float32), length=1.0)

# For the labels, the camera frame will coincide with the origin so the label is easy to place
plot += k3d.text('camera', position=[0, 0, 0], color=0xff0000, size=.5)
# Regarding the lidar and radar, the position for the label can be taken from the last column of the homogeneous matrix
lidar_label_pos = T_camera_lidar[:3,3]
radar_label_pos = T_camera_radar[:3,3]
plot += k3d.text('lidar', position=lidar_label_pos, color=0x00ff00, size=.5)
plot += k3d.text('radar', position=radar_label_pos, color=0x0000ff, size=.5)

# raise NotImplementedError()

plot.display()

In [ ]:
# CELL REF: C1.2c
# DO NOT DELETE THIS CELL!

### Q 01.1 Camera axes conventions
Within robotics, a usual convention of sensor frames is `x` pointing forward, `y` to the left and `z` to the top.
We used these conventions for the LiDAR and the radar measurement frames.
We also follow the typical convention for camera frames.

In which directions do the axes `x`, `y` and `z` of the camera frame correspond to?

In [ ]:
# CELL REF: C1.2d
pointing_directions_to_choose_from = {
    "viewing_direction",
    "inverse_viewing_direction",
    "left_of_image",
    "right_of_image",
    "top_of_image",
    "bottom_of_image",
}

In [ ]:
# CELL REF: C1.2e
### A 01.1
x_pointing_towards = None  # set to one of pointing_directions_to_choose_from
y_pointing_towards = None  # set to one of pointing_directions_to_choose_from
z_pointing_towards = None  # set to one of pointing_directions_to_choose_from

# YOUR CODE HERE

x_pointing_towards = "right_of_image"
y_pointing_towards = "bottom_of_image"
z_pointing_towards = "viewing_direction"

# raise NotImplementedError()

In [ ]:
# CELL REF: C1.2f
assert x_pointing_towards in pointing_directions_to_choose_from
assert y_pointing_towards in pointing_directions_to_choose_from
assert z_pointing_towards in pointing_directions_to_choose_from

## 1.3 MP only - LiDAR point clouds and radar point clouds (3D)
Use `k3d.plot` to draw the LiDAR points of the measurements within the sequence in blue and the radar target points in red (and larger).
Also, draw the coordinate axes of both `lidar` and `radar` frame including text overlying their names.
The `lidar` frame should be the origin of the plot.

Draw the radial velocities of the radar sensor as `k3d.vectors` (cf. Practicum 3 MP).
Please be reminded that `Measurements.get_radar_compensated_radial_velocities()` represents the radial velocities in the `radar` frame and the plot is represented in the `lidar` frame.


In [ ]:
# CELL REF: C1.3a
import k3d
if COURSE_TYPE == 'MP' or RUN_ALL:
    # get first measurement of sequence
    measurements = next(iter(sequence))

    plot = k3d.plot(camera_auto_fit=True)

    # YOUR CODE HERE
    
    # This code will be mostly taken from Practicum 3
    
    # First we get the lidar radar points
    p_lidar = measurements.get_lidar_points()[:, 0:3]
    # Then we need to get the radar points in the lidar frame
    p_radar = measurements.get_radar_points()
    p_radar_lf = T_lidar_radar.dot(p_radar.T).T[:, 0:3]
    # Then we add them to the plot
    plot += k3d.points(positions=p_lidar, point_size=0.03, color=0x0000ff)
    plot += k3d.points(positions=p_radar_lf, point_size=0.3, color=0xff0000)
    
    # We add the coordinate axes for the lidar frame with its label
    plot += plot_axes(T_plotorigin_target = np.eye(4).astype(np.float32), length=1.0)
    plot += k3d.text('lidar', position = [0, 0, 0], color = 0x00ff00, size=.5)
    # And we do the same for the radar
    plot += plot_axes(T_plotorigin_target = T_lidar_radar.astype(np.float32), length = 1.0)
    radar_label_pos = T_lidar_radar[:3,3]
    plot += k3d.text('radar', position = radar_label_pos, color = 0x0000ff, size=.5)
    
    # Now we want to add the radial velocities of the radar sensor
    # First we get the measurements in the radar frame from the data
    compensated_radial_velocity =  measurements.get_radar_compensated_radial_velocities().astype(np.float32)
    # Then we calculate the unit vectors and multiply them by the respective radial velocity
    # Notice that the unit vector is calculated with the points in the radar frame since that is the only radial velocity value we have
    unit_vectors = np.zeros((p_radar.shape[0], 3))
    velocity_vectors = np.zeros((p_radar.shape[0], 3))
    for i in range(p_radar.shape[0]):
        unit_vectors[i, :] = [p_radar[i, 0] / np.sqrt(p_radar[i, 0]**2 + p_radar[i, 1]**2 + p_radar[i, 2]**2),
                              p_radar[i, 1] / np.sqrt(p_radar[i, 0]**2 + p_radar[i, 1]**2 + p_radar[i, 2]**2),
                              p_radar[i, 2] / np.sqrt(p_radar[i, 0]**2 + p_radar[i, 1]**2 + p_radar[i, 2]**2)] 
        velocity_vectors[i, :] = unit_vectors[i, :] * compensated_radial_velocity[i]
    # Finally we add the vectors to the plot starting from the radar points in the lidar frame
    plot += k3d.vectors(origins=p_radar_lf.astype(np.float32), vectors=velocity_vectors.astype(np.float32), color=0xff0000)
    
#     raise NotImplementedError()
    plot.display()

In [ ]:
# CELL REF: C1.3b
if COURSE_TYPE == 'MP' or RUN_ALL:
    point_clouds = [o.positions for o in plot.objects if o.type == "Points"]
    assert len(point_clouds) == 2

## 1.4 MP only - LiDAR point clouds in the camera image
Now, project the LiDAR point cloud into the camera image of the first frame of the sequence using the `project_points` function you implemented in Practicum 1.
Use `cv2.circle` to draw **filled circles in red** `(0, 0, 255)`, `radius=3` and `thickness=-1`, so we can check the output.
Make sure you only draw visible points into the camera image.

In [ ]:
# CELL REF: C1.4a
import numpy as np
import cv2

from common.visualization import showimage
from ipynb.fs.defs.practicum1 import project_points
if COURSE_TYPE == 'MP' or RUN_ALL:
    image_draw = np.zeros((1216, 1936, 3))  # overwrite me
    color_red = (0, 0, 255)

    # YOUR CODE HERE
    
    # Get the measurements from the first frame
    measurements = sequence[start_index]
    
    # I get the camera image and projection matrix from the data from the data
    image_draw = measurements.get_camera_image()
    projection_matrix = measurements.get_camera_projection_matrix()
    
    # I also get the lidar points and ransform them in the camera frame
    p_lidar = measurements.get_lidar_points()
    p_lidar_cf = T_camera_lidar.dot(p_lidar.T).T
    
    # I now project the points in the camera image
    uvs = project_points(projection_matrix, p_lidar_cf)
    # I reduce the array to only include the lidar points visible in the camera image
    uvs_reduced = uvs[uvs[:, 0] >= 0]
    uvs_reduced = uvs_reduced[uvs_reduced[:, 0] < image_draw.shape[1]]
    uvs_reduced = uvs_reduced[uvs_reduced[:, 1] >= 0]
    uvs_reduced = uvs_reduced[uvs_reduced[:, 1] < image_draw.shape[0]]
    
    # Finally, I draw the circles in the image
    for uv in uvs_reduced:
         cv2.circle(image_draw, tuple(uv.astype(int)), 3, color_red, -1)
    
#     raise NotImplementedError()

    showimage(image_draw)

In [ ]:
# CELL REF: C1.4b
if COURSE_TYPE == 'MP' or RUN_ALL:
    color_red = (0, 0, 255)
    assert image_draw.shape == (1216, 1936, 3)
    assert np.all(image_draw[806, 7] == color_red)

Projected coordinates are conventionally denoted by `u` and `v`.

1. Which camera frame axes do they correspond to?
2. In which direction does a point with increasing `u` wander?
3. In which direction doeas a point with increasing `v` wander?

In [ ]:
# CELL REF: C1.4c
if COURSE_TYPE == 'MP' or RUN_ALL:
    axis_corresponding_to_u = None  # choose from "x", "y", "z"
    axis_corresponding_to_v = None  # choose from "x", "y", "z"
    direction_wandering_with_increasing_u = None  # choose from "up", "down", "left", "right"
    direction_wandering_with_increasing_v = None  # choose from "up", "down", "left", "right"

    # YOUR CODE HERE
    
    axis_corresponding_to_u = "x"  # choose from "x", "y", "z"
    axis_corresponding_to_v = "y"  # choose from "x", "y", "z"
    direction_wandering_with_increasing_u = "right"  # choose from "up", "down", "left", "right"
    direction_wandering_with_increasing_v = "down"  # choose from "up", "down", "left", "right"
    
#     raise NotImplementedError()

In [ ]:
# CELL REF: C1.4d
if COURSE_TYPE == 'MP' or RUN_ALL:
    assert axis_corresponding_to_u in {"x", "y", "z"}
    assert axis_corresponding_to_v in {"x", "y", "z"}
    assert direction_wandering_with_increasing_u in {"up", "down", "left", "right"}
    assert direction_wandering_with_increasing_v in {"up", "down", "left", "right"}

How many LiDAR points are being projected **onto** the image?
Please fill `n_lidar_points_within_camera_image` below with the proper value.

In [ ]:
# CELL REF: C1.4e
if COURSE_TYPE == 'MP' or RUN_ALL:
    n_lidar_points_within_camera_image = 0  # replace me

    # YOUR CODE HERE
    
    n_lidar_points_within_camera_image = uvs_reduced.shape[0]
    
#     raise NotImplementedError()

In [ ]:
# CELL REF: C1.4f
if COURSE_TYPE == 'MP' or RUN_ALL:
    # if below assertion fails, please make sure that your project_points implementation actually
    # uses rounded values
    assert n_lidar_points_within_camera_image == 20382, n_lidar_points_within_camera_image

## 1.5 MP only - Radar measurements in the camera image (video)
Now, create a video which displays the radar measurement point cloud of each frame of the sequence as circles over the camera image.
Please draw **filled circles in yellow** `(0, 255, 255)`, `radius=20` and `thickness=-1`, so we can check the output.
You don't have to draw radial velocities.

In [ ]:
# CELL REF: C1.5a
from common.visualization import create_animation
from IPython.core.display import HTML
from tqdm.notebook import tqdm
video = None
if COURSE_TYPE == 'MP' or RUN_ALL:
    color_yellow = (0, 255, 255)
    radius = 20
    images_draw = []  # append images here

    # YOUR CODE HERE
    
    # We iterate over all the time instants in the sequence
    for measurements in sequence:
        # First we get the image and the projection matrix for the single frame
        image_draw = measurements.get_camera_image()
        projection_matrix = measurements.get_camera_projection_matrix()
        # Now we do the same procedure as before only with radar points instead of lidar for the single frame
        p_radar = measurements.get_radar_points()
        p_radar_cf = T_camera_radar.dot(p_radar.T).T
        uvs = project_points(projection_matrix, p_radar_cf)
        uvs_reduced = uvs[uvs[:, 0] >= 0]
        uvs_reduced = uvs_reduced[uvs_reduced[:, 0] < image_draw.shape[1]]
        uvs_reduced = uvs_reduced[uvs_reduced[:, 1] >= 0]
        uvs_reduced = uvs_reduced[uvs_reduced[:, 1] < image_draw.shape[0]]
        # Now we draw the radar points in the single frame
        for uv in uvs_reduced:
            cv2.circle(image_draw, tuple(uv.astype(int)), radius, color_yellow, -1)
        # Finally we append the modified frame to the list for the animation
        images_draw.append(image_draw)

#     raise NotImplementedError()

    # we use create_animation to show the video inline
    # and yes: it takes some time
    anim = create_animation(images_draw)
    video = anim.to_html5_video()
HTML(video)

In [ ]:
# CELL REF: C1.5b
if COURSE_TYPE == 'MP' or RUN_ALL:
    assert len(images_draw) == 116, len(images_draw)
    assert anim.save_count == 116, len(anim.save_count)

## 1.6 IV only - Vehicle pose over time
The dataset also offers the vehicle pose for each frame with respect to a world static frame. The function `measurements.get_T_world_radar()` gives the transformation of the vehicle-mounted radar frame to a world static point (think: origin of `world` is the pose of the radar frame when ignition was turned on, so the car might have traveled a couple of kilometers from there).

Your next task is to create a `k3d` plot which denotes the radar frame locations over the sequence.
Please introduce a frame `radar0` which represents the pose of the radar frame at the beginning of the sequence.
This frame should be the origin of the plot, denoted by a larger axes (see the `length` parameter of `plot_axes`).

Now create the 3D plot showing the trajectory of the radar frame with respect to `radar0`.
Make sure you draw exactly one axes per timestep.

In [ ]:
# CELL REF: C1.6a
from common.k3d_helpers import plot_axes
if COURSE_TYPE == 'IV' or RUN_ALL:
    dataset = Dataset()
    sequence = dataset.get_custom_sequence(start_index, end_index)

    plot = k3d.plot(camera_auto_fit=False)

    # YOUR CODE HERE
    raise NotImplementedError()

    plot.display()

In [ ]:
# CELL REF: C1.6b
if COURSE_TYPE == 'IV' or RUN_ALL:
    axes_objects = [o for o in plot.objects if o.type == "Vectors"]
    assert len(axes_objects) in {
        n_timesteps,
        n_timesteps + 1,  # start might be drawn as small and large axes over each other
    }

**IV only**

How many meters has the car travelled from start to the end of the sequence?
Approximate with intermediate trajectory points.
Consider movement along x, y, and z.

Please set the value of `distance_travelled_m` below.

In [ ]:
# CELL REF: C1.6c
if COURSE_TYPE == 'IV' or RUN_ALL:
    distance_travelled_m = 0.0  # replace me with the proper distance

    # YOUR CODE HERE
    raise NotImplementedError()
    print(distance_travelled_m)

In [ ]:
# CELL REF: C1.6d
if COURSE_TYPE == 'IV' or RUN_ALL:
    # DO NOT DELETE THIS CELL!
    pass

## 1.7 Annotated objects
The dataset comes with manually annotated objects, which we also call (ground truth) labels.
It includes the object class, its transformation with the camera frame `T_cam_object` and its extent in object frame (length, width, height):
* length: along object x axis
* width: along object y axis
* height: along object z axis

The origin of the object is at the bottom center (see `get_corners_object` in practicum 1).

Let's have a look at an example annotation.

In [ ]:
# CELL REF: C1.7a
from common.k3d_helpers import plot_box

dataset = Dataset()
sequence = dataset.get_custom_sequence(start_index, end_index)
measurements = next(iter(sequence))
labels_camera = measurements.get_labels_camera()
len(labels_camera), labels_camera[1]

In [ ]:
# CELL REF: C1.7b
import numpy as np

# Let's make sure the implementation of get_T_cam_object() does the right thing
# Otherwise you need to fix your implementation of get_T_cam_object() within practicum 1
# get_T_cam_object() from practicum 1 is being called in get_labels_camera()
assert np.allclose(
    labels_camera[1]["T_cam_object"],
    np.array(
        [
            [-0.7481051, -0.66340834, 0.0151049, 7.933778],
            [0.06748124, -0.09870181, -0.9928264, 3.0391772],
            [0.66014016, -0.7417193, 0.1186069, 14.156578],
            [0.0, 0.0, 0.0, 1.0],
        ]
    ),
)

As you can see, the first frame of the sequence has 13 annotation, and the label with index 1 is a Pedestrian with a height of about 1.69m (`labels_camera[1]["extent_object"][2]`).

### Plot annotated objects

We created a helper function for you which adds a 3D representation of a label to a `k3d` plot. The function `plot_box(plot, label_camera, T_target_camera)` takes the information from the `label_camera` dict, and uses the transform `T_target_camera` to transform the labels originally given in the `camera` frame to the `target` frame.
`target` frame corresponds to the origin of the plot.

Now, please create a `k3d.plot` and visualize all the labels of the first frame within the sequence in the `lidar` frame.
We also plot the axes of the `lidar` frame.

In [ ]:
# CELL REF: C1.7c
plot = k3d.plot()
plot += plot_axes(np.eye(4, dtype=np.float32))  # this should represent the LiDAR frame

# YOUR CODE HERE

for i in range(len(labels_camera)):
    plot_box(plot, labels_camera[i], T_lidar_cam)    

# raise NotImplementedError()

plot.display()

In [ ]:
# CELL REF: C1.7d
axes_objects = [o for o in plot.objects if o.type == "Vectors"]
assert len(axes_objects) == 13 + 1, len(axes_objects)

### Colors of object classes
Which class do the different colors of the bounding boxes represent?

In [ ]:
# CELL REF: C1.7e
available_colors = {"red", "blue", "green", "yellow", "gray", "magenta", "orange"}
available_classes = {"Car", "Cyclist", "DontCare", "Pedestrian"}

In [ ]:
# CELL REF: C1.7f

# set the dictionary to map from available colors to available classes
# make sure you only use colors which are actually present in the visualization
# make sure to only use classes which are actually present in the visualization
color_represents = {
    "red": None,  # set me to proper class
    # extend me with other color to class mappings
}

# YOUR CODE HERE

color_represents = {
    "red": "Car",
    "blue": "Cyclist",
    "green": "Pedestrian",
    "gray": "DontCare"
}

# raise NotImplementedError()

In [ ]:
# CELL REF: C1.7g
assert set(color_represents.keys()).issubset(available_colors)
assert set(color_represents.values()).issubset(available_classes)

The function `get_corners_object` gets the 8 corner points of a label dictionary as homogeneous points within the object frame.

In [ ]:
# CELL REF: C1.7h
from ipynb.fs.defs.practicum1 import get_corners_object

label_camera = labels_camera[1]
corners_object = get_corners_object(label_camera["extent_object"])
corners_object

### Project object corners
Now, your task is to draw the projection of those 8 points of `label_camera` as *filled* circles (radius >= 3) into the camera image.
You can choose any color, as long its distinguishable from the image content.
All points need to be of the same color.

In [ ]:
# CELL REF: C1.7i
image_draw = measurements.get_camera_image()

# YOUR CODE HERE

projection_matrix = measurements.get_camera_projection_matrix()

for i in range(len(labels_camera)):
    label_camera = labels_camera[i]
    corners_object = get_corners_object(label_camera["extent_object"])
#     print(corners_object)
    
    T_cam_object = label_camera["T_cam_object"]
    corners_object_cf = T_cam_object.dot(corners_object.T).T
#     print(object_position)
    
#     corners_object_cf = T_camera_lidar.dot((corners_object + object_position).T).T
#     print(corners_object + object_position)
    
    uvs = project_points(projection_matrix, corners_object_cf)
#     print(uvs)
    
    for uv in uvs:
        cv2.circle(image_draw, tuple(uv.astype(int)), 5, (0, 255, 0), -1)

# raise NotImplementedError()
showimage(image_draw)

In [ ]:
# CELL REF: C1.7j
# DO NOT DELETE THIS CELL!

## 1.8 Bounding Box
Now, create a function which creates a `BoundingBox` object (see Practicum 1) from the `objects_dict` (`label_camera`).
We use the helper function `draw_bbox_to_image` to draw the `BoundingBox` object to the image `image_draw`. 

In [ ]:
# CELL REF: C1.8a
from common.visualization import draw_bbox_to_image
from practicum1.BoundingBox import BoundingBox

print(label_camera["2d_bbox"])

def get_bounding_box_from_object(object_dict, projection_matrix):

    bbox = BoundingBox(1, 2, 3, 4)  # set me to the right values

    # YOUR CODE HERE
    
    # Get the object dimension and position from the dictionary
    extent_object = object_dict["extent_object"]
    T_cam_object = object_dict["T_cam_object"]
    
    # Rotate the dimension to be compliant to the camera frame
    R_cam_object = T_cam_object[0:3, 0:3]
    extent_object_cf = R_cam_object.dot(extent_object)
    
    # Calculate the corners of the box in the 3D space
    corners = []
    corners.append([T_cam_object[0, 3] - extent_object_cf[0]/2, T_cam_object[1, 3],                       T_cam_object[2, 3]])
    corners.append([T_cam_object[0, 3] + extent_object_cf[0]/2, T_cam_object[1, 3],                       T_cam_object[2, 3]])
    corners.append([T_cam_object[0, 3] - extent_object_cf[0]/2, T_cam_object[1, 3] + extent_object_cf[1], T_cam_object[2, 3]])
    corners.append([T_cam_object[0, 3] + extent_object_cf[0]/2, T_cam_object[1, 3] + extent_object_cf[1], T_cam_object[2, 3]])
    corners = np.array(corners)

    # Project the points in 2D space
    corners_aug = np.hstack((corners, np.ones((corners.shape[0], 1))))
    corners_2d = project_points(projection_matrix, corners_aug)

    # Create the bounding boxes from the corners
    if corners_2d[1, 0] < corners_2d[2, 0]:
        bbox = BoundingBox(corners_2d[2, 1].astype(np.int32), corners_2d[1, 0].astype(np.int32), corners_2d[1, 1].astype(np.int32), corners_2d[2, 0].astype(np.int32), from_corners=True)
    else:
        bbox = BoundingBox(corners_2d[2, 1].astype(np.int32), corners_2d[2, 0].astype(np.int32), corners_2d[1, 1].astype(np.int32), corners_2d[1, 0].astype(np.int32), from_corners=True)
    
#     raise NotImplementedError()
    assert isinstance(bbox, BoundingBox)
    return bbox

# Here it made more sense to me to print all the bounding boxes in the image, therefore I added an iteration for all the objects
for label_camera in labels_camera:
    bbox = get_bounding_box_from_object(label_camera, measurements.get_camera_projection_matrix())
    draw_bbox_to_image(image_draw, bbox, thickness=1)
showimage(image_draw)

In [ ]:
# CELL REF: C1.8b
# DO NOT REMOVE THIS CELL!

## 1.9 Ground plane visualization
The road surface in front of the vehicle can be represented as a flat ground plane.
A representation of a plane is `[a, b, c, d]` and the plane is defined by all points `[x, y, z]` which suffice the equation $ax + by + cz + d = 0$

E.g., for the first frame of the sequence, the plane coefficients in `lidar` frame are:

In [ ]:
# CELL REF: C1.9a
plane_lidar = np.array([0.00312662, 0.0069831, 0.99997073, 1.58132066])
plane_lidar

We can *sample* points on the plane by fixing an x and y value and calculating the corresponding z value (or any other combination).

### Meshgrid
Now, find the Z points which reside on the plane on a rectangular grid of x and y values (`x_min`..`x_max`, `y_min`..`y_max`) with distance of `x_step = y_step = 0.5`m.
See `np.linspace`/`np.arange` and `np.meshgrid`.
The maxima are defined inclusive, e.g., along the x-axis there should be 41 points.

In [ ]:
# CELL REF: C1.9b
# do not change these
x_min = 2
x_max = 22
y_min = -10
y_max = 10
x_step = 0.5
y_step = 0.5


def get_plane_meshgrid(plane_model, x_min, x_max, y_min, y_max, x_step, y_step):
    a, b, c, d = plane_model

    # number of elements in x and y direction
    n_x = int((x_max - x_min) / x_step) + 1  # +1 to account for x_max being inclusive
    n_y = int((y_max - y_min) / y_step) + 1  # +1 to account for y_max being inclusive

    # YOUR CODE HERE
    
    # I first define all the values for x and y
    x = np.arange(x_min, x_max + x_step, x_step)
    y = np.arange(y_min, y_max + y_step, y_step)
    # Then I create the meshgrid with those values
    X, Y = np.meshgrid(x, y)
    # Finally I calculate the value of Z using the plane equation
    Z = (a * X + b * Y + d) / (- c)
    
#     raise NotImplementedError()

    assert X.shape == (n_y, n_x)
    assert Y.shape == (n_y, n_x)
    assert Z.shape == (n_y, n_x)

    return X, Y, Z


Xs_lidar, Ys_lidar, Zs_lidar = get_plane_meshgrid(plane_lidar, x_min, x_max, y_min, y_max, x_step, y_step)
# let's stack the values so we have XYZ values of all points on the mesh
XYZs_lidar = np.vstack([Xs_lidar.ravel(), Ys_lidar.ravel(), Zs_lidar.ravel()]).T
XYZs_lidar.shape

In [ ]:
# CELL REF: C1.9c
assert XYZs_lidar.shape == (1681, 3)

### 3D mesh plot
Now, plot the sampled `XYZs` points on the grid with `k3d.points` within the LiDAR frame as red dots. 
Please draw a `k3d.mesh` of the plane.
You can feed `k3d.mesh` with `XYZs` after applying a Delaunay triangulation from `matplotlib.tri.Triangulation`.
See also [K3D-jupyter mesh example](https://k3d-jupyter.org/basic_functionality/Mesh.html).
Setting `attribute` and/or `opacity` might come in handy.

**MP only**:
Please also plot the LiDAR points of `start_index` alongside, so you can visually verify that you've been implementing the right thing.

In [ ]:
# CELL REF: C1.9d
plot = k3d.plot()
plot += plot_axes(np.eye(4, dtype=np.float32))  # LiDAR frame

# YOUR CODE HERE

# First I plot the sampled XYZs points
plot += k3d.points(positions=XYZs_lidar.astype(np.float32), point_size=0.1, color=0xff0000)

# Then I import the triangulation function
from matplotlib.tri import Triangulation
# Then I get the triangle vertices by triangulating the flattened arrays
indices = Triangulation(Xs_lidar.flatten(), Ys_lidar.flatten()).triangles.astype(np.uint32)
# Finally I plot the mesh
plt_mesh = k3d.mesh(XYZs_lidar.astype(np.float32), indices, attribute=Zs_lidar)
plot += plt_mesh

# Finally I plot the lidar points
measurements = sequence[start_index]
p_lidar = measurements.get_lidar_points()
plot += k3d.points(positions=p_lidar[:, 0:3].astype(np.float32), point_size=0.03, color=0x0000ff)

# raise NotImplementedError()

plot.display()

In [ ]:
# CELL REF: C1.9e
points_objects = [o for o in plot.objects if o.type == "Points"]
if COURSE_TYPE == 'MP' or RUN_ALL:
    assert len(points_objects) == 2  # LiDAR and grid
elif COURSE_TYPE == 'IV':
    assert len(points_objects) in {1, 2}  # at least grid, LiDAR optional
  
mesh_objects = [o for o in plot.objects if o.type == "Mesh"]
assert len(mesh_objects) == 1

## 1.10 Precomputed ground planes and trajectory in world static frame
We provide you **precomputed ground planes** in the camera frame and with the **trajectory** of the camera frame within a world static frame.

### Ground planes
You can obtain precomputed ground planes given in the camera frame via `Measurements.get_ground_plane()`.
It follows the `[a, b, c, d]` convention mentioned in the previous notebook (but is given in the **camera frame**).

### Trajectory
`T_newworld_camera` (via `Measurements.get_T_newworld_camera()`) represents the pose of the camera over time for the world-static frame `newworld`.
`newworld` has its origin at the pose of the radar sensor during frame `1430`, i.e. at frame `1430`, the `T_newworld_cam` corresponds to `T_radar_cam`.
We called it *`newworld`* to differentiate from the `world` frame in `Measurements.get_T_world_radar()`.

### Your Task
#### Plot camera axes in `newworld`
Use a `k3d.plot` to visualize the trajectory of the `camera` frame within the `newworld` frame as axes (`plot_axes()`) for every 10th frame within the sequence.
The getter is `Measurements.get_T_newworld_camera()`
Also plot the axes of the `newworld` frame.
Use `k3d.text` to name the frame `newworld`, and the index (1430, 1440, ...) for the trajectoy axes.

#### Plot precomputed ground planes
In addition, please plot the ground plane of each 10th frame as a mesh in the `newworld` frame.
Use distinct colors, like `from common.visualization import colors_qualitative_k3d`.
Consider, that the ground plane model is given in **camera frame** for each frame, so sample y values for a grid of x and z values ([`x_min`..`x_max`] and [`z_min`..`z_max`]), as `y` values of the plane (within the camera frame) are nearly constant within the camera frame.
Or use `transform_plane()` below and sample in `lidar` frame in `x` and `y` as we did above.

#### Example output
The resulting k3d plot should look similar to this:
![Example k3d plot](fa_01a_ground_plane_trajectories.png)

In [ ]:
# CELL REF: C1.10a
import k3d
from common.sequence_loader import Dataset
from common.visualization import colors_qualitative_k3d
from common.k3d_helpers import plot_axes

# you might find the below function handy to transform plane coefficients between different frames.
# E.g., when you want to reuse get_plane_meshgrid
# which samples along x and y dimension e.g., in LiDAR frame,
# while the plane coefficients via Measurements.get_ground_plane() are given in camera frame,
# where sampling along x and z axis would be the thing to do.
def transform_plane(plane_model_source, T_source_target):
    """
    Transforms plane coefficients [a,b,c,d]_source from plane_model_source represented in source frame
    into plane coefficients [a,b,c,d]_target represented in target frame via T_source_target.

    it works due to the identity of
         [a,b,c,d]_source * p_source                      = 0  # p_source a point that lies in the source plane 
    <=>  [a,b,c,d]_source * (T_source_target  * p_target) = 0  # transform target points into source frame
    <=> ([a,b,c,d]_source *  T_source_target) * p_target  = 0  # associativity of ()
    <=>  [a,b,c,d]_target                     * p_target  = 0  # define [a,b,c,d]_target by content of bracket
    """
    return plane_model_source.reshape(1, 4).dot(T_source_target)[0]  # unreshape


start_index = 1430
end_index = 1545

dataset = Dataset()
sequence = dataset.get_custom_sequence(start_index, end_index)

plot = k3d.plot()

# boundaries for ground plane in *LiDAR* frame
x_min = 2.0
x_max = 22.0
y_min = -5.0
y_max = 5.0
# steps every 5 m should be sufficient as the ground plane is flat
# so the steps along the x axis are 2.0, 7.0, 12.0, 17.0, 22.0
# and for y axis -5.0, 0.0, 5.0
x_step = 5.0
y_step = 5.0

plot += k3d.text("newworld", position=[0.0, 0.0, 0.0], color=0x00FF00, size=0.5)
plot += plot_axes(np.eye(4, dtype=np.float32))  # newworld

# YOUR CODE HERE

# First I define the color array and initialize a counter for the colors
colors = colors_qualitative_k3d
j = 0

# Iterate to every 10th frame
for i in np.arange(start_index, end_index, 10):
    
    # Get the measurements for this frame
    measurements = sequence[i]
    
    # Plot label and axes for the camera
    T_newworld_camera = measurements.get_T_newworld_camera()
    plot += k3d.text(str(i), position=T_newworld_camera[0:3, 3], color=colors[j], size=0.5)
    plot += plot_axes(T_newworld_camera.astype(np.float32))
    
    # Get the ground plane in camera frame from measuremts and trasnform it in lidar frame
    ground_plane_cf = measurements.get_ground_plane()
    ground_plane_lf = transform_plane(ground_plane_cf, T_cam_lidar)
    
    # Compute the XYZs values for the ground plane model
    Xs_lidar, Ys_lidar, Zs_lidar = get_plane_meshgrid(ground_plane_lf, x_min, x_max, y_min, y_max, x_step, y_step)
    XYZs_lidar = np.vstack([Xs_lidar.ravel(), Ys_lidar.ravel(), Zs_lidar.ravel()]).T
    
    # Define the transformation from newworld to lidar frame by combining the camera transformations
    T_newworld_lidar = T_newworld_camera.dot(T_cam_lidar)
    
    # Transform the XYZs values from lidar frame to newworld frame with this new transformation
    XYZs_lidar_aug = np.hstack((XYZs_lidar, np.ones((XYZs_lidar.shape[0], 1))))
    XYZs_lidar = T_newworld_lidar.dot(XYZs_lidar_aug.T).T
    XYZs_lidar = XYZs_lidar[:, 0:3]
    
    # Finally, draw the mesh
    indices = Triangulation(Xs_lidar.flatten(), Ys_lidar.flatten()).triangles.astype(np.uint32)
    plt_mesh = k3d.mesh(XYZs_lidar.astype(np.float32), indices, color=colors[j], opacity=0.5)
    plot += plt_mesh
    
    # Increment the color counter
    j += 1
    
# raise NotImplementedError()
plot.display()

In [ ]:
# CELL REF: C1.10b
vector_objects = np.asarray([o for o in plot.objects if o.type == "Vectors"])
assert len(vector_objects) == 13, len(vector_objects)  # 12 camera frames and newworld frame

mesh_objects = [o for o in plot.objects if o.type == "Mesh"]
assert len(mesh_objects) == 12, len(mesh_objects)

# Wrap up
That's it.
You've shown us that you have the prerequisites to do actual processing on the dataset.
Great job!